In [ ]:
 from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/My Drive

/content/drive/My Drive


# **Question 2**

In [ ]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 13.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3162680 sha256=d28dab122dd9a5a3f6ed76a471070f8c494c0e09cdba86c09c6ecf9e080d912d
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [ ]:
import pandas as pd
import numpy as np
import random
from surprise import KNNBasic
from surprise import Dataset
from surprise import Reader
from scipy.stats import pearsonr
from collections import defaultdict
import time

In [ ]:
ratings_df = pd.read_csv('movielens_ratings.csv') #Read dataset
ratings_df_600 = ratings_df.head(600000) #Get first 600000 rows of dataset

# **Question 3**

In [ ]:
# Choose 10 random users
random_users = random.sample(list(ratings_df_600['user_id'].unique()), 10)

# Randomly remove one rating for each user
for user in random_users:
    user_ratings = ratings_df_600[ratings_df_600['user_id'] == user]
    random_rating = random.choice(user_ratings.index)
    ratings_df_600_q3 = ratings_df_600.drop(random_rating)

In [ ]:
# Create a Surprise Dataset from the modified ratings DataFrame
reader = Reader(rating_scale=(1, 5))
data_sup_q3 = Dataset.load_from_df( ratings_df_600_q3[['user_id', 'movie_id', 'rating']], reader)

In [ ]:
# Build the full training set
trainset_userbased = data_sup_q3.build_full_trainset()

In [ ]:
# Create the KNNBasic model
model = KNNBasic( sim_options={'user_based': True})

In [ ]:
# Train the model on the full training set
model.fit(trainset_userbased)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [ ]:
# Initialize lists to store actual and predicted ratings
actual_ratings = []
predicted_ratings = []

In [ ]:
# Iterate over the random users
for user in random_users:
    # Get the user's ratings
    user_ratings =  ratings_df_600_q3[ ratings_df_600_q3['user_id'] == user]

    # Get the item ID of the removed rating
    removed_item = user_ratings['movie_id'].values[0]

    # Get the actual rating for the removed item
    actual_rating = user_ratings['rating'].values[0]
    actual_ratings.append(actual_rating)

    # Predict the rating for the removed item based on the neighbors
    predicted_rating = model.predict(trainset_userbased.to_inner_uid(user), trainset_userbased.to_inner_iid(removed_item),
                                     verbose=False).est
    predicted_ratings.append(predicted_rating)

In [ ]:
# Convert actual_ratings and predicted_ratings to numpy arrays
actual_ratings = np.array(actual_ratings)
predicted_ratings = np.array(predicted_ratings)
print("actual_ratings:",actual_ratings )
print("predicted_ratings:",predicted_ratings)

actual_ratings: [5 3 5 4 4 5 1 4 3 5]
predicted_ratings: [4.16166211 3.80267872 4.31707624 4.32437825 3.60749584 3.5614604
 4.14093946 3.8452836  3.19463671 3.28890021]


In [ ]:
# Calculate MAE and RMSE
mae = np.mean(np.abs(predicted_ratings - actual_ratings))
rmse = np.sqrt(np.mean((predicted_ratings - actual_ratings) ** 2))

print("MAE:", mae)
print("RMSE:", rmse)

MAE: 0.9680754743720396
RMSE: 1.3037388217701855


# **Question 4**

# **based on k**

In [ ]:
# Initialize variables for best k and best MAE/RMSE
best_k = None
best_mae_k = float('inf')
best_rmse_k = float('inf')

In [ ]:
# Try different values of k
for k in [5, 15, 30, 50, 100]:
    # Create the KNNBasic model with the current k value
    model = KNNBasic(k=k, sim_options={'user_based': True})

    # Train the model on the full training set
    model.fit(trainset_userbased)

    # Initialize lists to store actual and predicted ratings
    actual_ratings = []
    predicted_ratings = []

    # Iterate over the random users
    for user in random_users:
        # Get the user's ratings
        user_ratings =  ratings_df_600_q3[ ratings_df_600_q3['user_id'] == user]

        # Get the item ID of the removed rating
        removed_item = user_ratings['movie_id'].values[0]

        # Get the actual rating for the removed item
        actual_rating = user_ratings['rating'].values[0]
        actual_ratings.append(actual_rating)

        # Predict the rating for the removed item based on the neighbors
        predicted_rating = model.predict(trainset_userbased.to_inner_uid(user), trainset_userbased.to_inner_iid(removed_item),
                                         verbose=False).est
        predicted_ratings.append(predicted_rating)

    # Convert actual_ratings and predicted_ratings to numpy arrays
    actual_ratings = np.array(actual_ratings)
    predicted_ratings = np.array(predicted_ratings)

    # Calculate MAE and RMSE
    mae = np.mean(np.abs(predicted_ratings - actual_ratings))
    rmse = np.sqrt(np.mean((predicted_ratings - actual_ratings) ** 2))

    # Print the MAE and RMSE for the current k
    print(f"k = {k}")
    print("MAE:", mae)
    print("RMSE:", rmse)
    print("-----------------------------")

    # Check if current k gives lower MAE and RMSE
    if mae < best_mae_k and rmse < best_rmse_k:
        best_k = k
        best_mae_k = mae
        best_rmse_k = rmse

# Print the best k value, MAE, and RMSE
print("Best k:", best_k)
print("Best MAE:", best_mae_k)
print("Best RMSE:", best_rmse_k)

Computing the msd similarity matrix...
Done computing similarity matrix.
k = 5
MAE: 0.84156496501267
RMSE: 1.177584917362434
-----------------------------
Computing the msd similarity matrix...
Done computing similarity matrix.
k = 15
MAE: 0.9649454044344192
RMSE: 1.3165514480491627
-----------------------------
Computing the msd similarity matrix...
Done computing similarity matrix.
k = 30
MAE: 0.9641604870119991
RMSE: 1.3154927844717865
-----------------------------
Computing the msd similarity matrix...
Done computing similarity matrix.
k = 50
MAE: 0.9773202613053666
RMSE: 1.3014185580267446
-----------------------------
Computing the msd similarity matrix...
Done computing similarity matrix.
k = 100
MAE: 1.0221232592363387
RMSE: 1.3123826253126274
-----------------------------
Best k: 5
Best MAE: 0.84156496501267
Best RMSE: 1.177584917362434


##Calcualte Precsion , Recall and F1-score for user-based Collaborarive Filtering based on k.

In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
from surprise import KNNBasic
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split

# Load MovieLens ratings dataset
ratings_df = pd.read_csv('movielens_ratings.csv')
ratings_df = ratings_df.sample(frac=0.3, random_state=42)

# Create Surprise Dataset
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings_df[['user_id', 'movie_id', 'rating']], reader)

# Train-test split
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# Initialize lists to store results for different K values
results = []

# Iterate over different K values
for k in [5, 15, 30, 50, 100]:
    # Create the KNNBasic model with the current k value
    model = KNNBasic(k=k, sim_options={'user_based': True})

    # Train the model on the full training set
    model.fit(trainset_userbased)

    # Initialize lists to store relevant items and recommended items
    relevant_items = []
    recommended_items = []

    # Iterate over the test set
    for user_id, movie_id, rating in testset:
        # Get the relevant items (items rated 4 or higher by the user in the test set)
        if rating >= 4:
            relevant_items.append(movie_id)

        # Predict the rating for the item based on the neighbors
        predicted_rating = model.predict(user_id, movie_id).est

        # Recommend the item if the predicted rating is 4 or higher
        if predicted_rating >= 4:
            recommended_items.append(movie_id)

    # Calculate precision and recall
    intersection = len(set(recommended_items) & set(relevant_items))
    precision = intersection / len(recommended_items) if len(recommended_items) > 0 else 0.0
    recall = intersection / len(relevant_items)
    F1 = 2 * (precision * recall) / (precision + recall)

    # Store the results for this K value
    results.append((k, precision, recall, F1))

# Sort the results based on F1-score in descending order
results.sort(key=lambda x: x[3], reverse=True)

# Print precision, recall, and F1-score for each K value
for k, precision, recall, F1 in results:
    print("K =", k)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1-Score:", F1)
    print()

# Print the best precision, recall, and F1-score value
best_k, best_precision, best_recall, best_F1 = results[0]
print("Best K =", best_k)
print("Best Precision:", best_precision)
print("Best Recall:", best_recall)
print("Best F1-Score:", best_F1)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
K = 5
Precision: 0.11787324317157252
Recall: 0.05165001161980014
F1-Score: 0.07182677547063102

K = 15
Precision: 0.10391152675721814
Recall: 0.04066930048803161
F1-Score: 0.05845877612376559

K = 30
Precision: 0.09254696643055128
Recall: 0.03491749941900999
F1-Score: 0.05070446300514637

K = 50
Precision: 0.08363665136888748
Recall: 0.030705321868463864
F1-Score: 0.04491946793591433

K = 100
Precision: 0.0732170894967358
Recall: 0.02573785730885429
F1-Score: 0.038087049973132725

Best K = 5
Best Precision: 0.11787324317157252
Best Recall: 0.05165001161980014
Best F1-Score: 0.07182677547063102


# **based on threshold**

In [ ]:
# Define the range of threshold intervals
threshold_intervals = [(0.5, 0.6), (0.6, 0.7), (0.7, 0.8), (0.8, 0.9), (0.9, 1.0)]

In [ ]:
# Initialize variables to store the best threshold, MAE, and RMSE
best_threshold = None
best_mae = float('inf')
best_rmse = float('inf')

In [ ]:
# Iterate over the threshold intervals
for threshold_interval in threshold_intervals:
    # Extract the lower and upper bounds of the threshold interval
    lower_bound, upper_bound = threshold_interval

    # Create a KNNBasic model with the current similarity threshold interval
    model = KNNBasic(sim_options={'user_based': True})

    # Train the model on the full dataset
    model.fit(trainset_userbased)

    # Initialize lists to store actual and predicted ratings
    actual_ratings = []
    predicted_ratings = []


    # Iterate over the random users
    for user in random_users:
        # Get the user's ratings
        user_ratings =  ratings_df_600_q3[ ratings_df_600_q3['user_id'] == user]

        # Get the item ID of the removed rating
        removed_item = user_ratings['movie_id'].values[0]

        # Get the actual rating for the removed item
        actual_rating = user_ratings[user_ratings['movie_id'] == removed_item]['rating'].values[0]

        # Calculate Pearson correlation between the current user and other users
        correlations = []
        for other_user in  ratings_df_600_q3['user_id'].unique():
            if other_user != user:
                other_user_ratings =  ratings_df_600_q3[ ratings_df_600_q3['user_id'] == other_user]
                common_ratings = user_ratings.merge(other_user_ratings, on='movie_id')
                if len(common_ratings) > 1 and np.var(common_ratings['rating_x']) > 0 and np.var(common_ratings['rating_y']) > 0:
                    correlation = pearsonr(common_ratings['rating_x'], common_ratings['rating_y'])[0]
                    correlations.append(correlation)

        # Filter neighbors based on similarity threshold
        filtered_neighbors = [ ratings_df_600_q3['user_id'].unique()[i] for i, correlation in enumerate(correlations)
                              if lower_bound <= correlation <= upper_bound]

        # Predict the rating based on the filtered neighbors
        predicted_rating = np.mean([model.predict(user, removed_item).est for user in filtered_neighbors])

        # Append the actual and predicted ratings to the lists
        actual_ratings.append(actual_rating)
        predicted_ratings.append(predicted_rating)

    # Convert actual_ratings and predicted_ratings to numpy arrays
    actual_ratings = np.array(actual_ratings)
    predicted_ratings = np.array(predicted_ratings)

    # Calculate MAE and RMSE
    mae = np.mean(np.abs(predicted_ratings - actual_ratings))
    rmse = np.sqrt(np.mean((predicted_ratings - actual_ratings) ** 2))

    # Check if current threshold interval gives lower MAE and RMSE
    if mae < best_mae and rmse < best_rmse:
        best_threshold = threshold_interval
        best_mae = mae
        best_rmse = rmse

    # Print the MAE and RMSE for the current threshold interval
    print(f"Threshold Interval: {threshold_interval}")
    print("MAE:", mae)
    print("RMSE:", rmse)
    print()

# Print the best threshold interval, MAE, and RMSE
print("Best Threshold Interval:", best_threshold)
print("Best MAE:", best_mae)
print("Best RMSE:", best_rmse)

Computing the msd similarity matrix...
Done computing similarity matrix.
Threshold Interval: (0.5, 0.6)
MAE: 0.8667245581443492
RMSE: 1.0350647228967733

Computing the msd similarity matrix...
Done computing similarity matrix.
Threshold Interval: (0.6, 0.7)
MAE: 0.8511456868920838
RMSE: 1.016250531717755

Computing the msd similarity matrix...
Done computing similarity matrix.
Threshold Interval: (0.7, 0.8)
MAE: 0.8837178613981941
RMSE: 1.0436629022800616

Computing the msd similarity matrix...
Done computing similarity matrix.
Threshold Interval: (0.8, 0.9)
MAE: 0.8807432752070318
RMSE: 1.044277213295718

Computing the msd similarity matrix...
Done computing similarity matrix.
Threshold Interval: (0.9, 1.0)
MAE: 0.8651130848756582
RMSE: 1.0303069170894592

Best Threshold Interval: (0.6, 0.7)
Best MAE: 0.8511456868920838
Best RMSE: 1.016250531717755


##Calculate Precsion , Recall and F1-score for user-based Collaborative Filtering based on threshold.

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import pearsonr
from surprise import KNNBasic

# Load MovieLens ratings dataset
ratings_df = pd.read_csv('movielens_ratings.csv')
ratings_df = ratings_df.sample(frac=0.3, random_state=42)

# Initialize variables for best threshold, precision, and recall
best_threshold = None
best_precision = 0.0
best_recall = 0.0

# Define the range of threshold intervals
threshold_intervals = [(0.5, 0.6), (0.6, 0.7), (0.7, 0.8), (0.8, 0.9), (0.9, 1.0)]

# Iterate over the threshold intervals
for threshold_interval in threshold_intervals:
    # Extract the lower and upper bounds of the threshold interval
    lower_bound, upper_bound = threshold_interval

    # Create a KNNBasic model with the current similarity threshold interval
    model = KNNBasic(sim_options={'user_based': True})

    # Train the model on the full dataset
    #model.fit(ratings_df)
    model.fit(trainset_userbased)

    # Initialize lists to store relevant items and recommended items
    relevant_items = []
    recommended_items = []

    # Iterate over the users
    for user in random_users:  # Replace "your_random_users_list" with your list of random users
        # Get the user's ratings
        user_ratings = ratings_df[ratings_df['user_id'] == user]

        # Get the item ID of the removed rating
        removed_item = user_ratings['movie_id'].values[0]

        # Get the actual rating for the removed item
        actual_rating = user_ratings[user_ratings['movie_id'] == removed_item]['rating'].values[0]

        # Calculate Pearson correlation between the current user and other users
        correlations = []
        for other_user in ratings_df['user_id'].unique():
            if other_user != user:
                other_user_ratings = ratings_df[ratings_df['user_id'] == other_user]
                common_ratings = user_ratings.merge(other_user_ratings, on='movie_id')
                if len(common_ratings) > 1 and np.var(common_ratings['rating_x']) > 0 and np.var(common_ratings['rating_y']) > 0:
                    correlation = pearsonr(common_ratings['rating_x'], common_ratings['rating_y'])[0]
                    correlations.append(correlation)

        # Filter neighbors based on similarity threshold
        filtered_neighbors = [ratings_df['user_id'].unique()[i] for i, correlation in enumerate(correlations)
                              if lower_bound <= correlation <= upper_bound]

        # Predict the rating based on the filtered neighbors
        predicted_rating = np.mean([model.predict(user, removed_item).est for user in filtered_neighbors])

        # Append the actual and predicted ratings to the lists
        relevant_items.append(removed_item)
        recommended_items.append(removed_item) if predicted_rating >= 4 else None

    # Calculate precision and recall
    intersection = len(set(recommended_items) & set(relevant_items))
    precision = intersection / len(recommended_items) if len(recommended_items) > 0 else 0.0
    recall = intersection / len(relevant_items)

    # Check if current threshold interval gives higher precision and recall
    if precision > best_precision and recall > best_recall:
        best_threshold = threshold_interval
        best_precision = precision
        best_recall = recall

    # Print the precision and recall for the current threshold interval
    print(f"Threshold Interval: {threshold_interval}")
    print("Precision:", precision)
    print("Recall:", recall)
    print()

# Print the best threshold interval, precision, and recall
print("Best Threshold Interval:", best_threshold)
print("Best Precision:", best_precision)
print("Best Recall:", best_recall)
print("F1-Score:", 2 * (best_precision * best_recall) / (best_precision + best_recall) )

Computing the msd similarity matrix...
Done computing similarity matrix.
Threshold Interval: (0.5, 0.6)
Precision: 1.0
Recall: 0.1

Computing the msd similarity matrix...
Done computing similarity matrix.


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Threshold Interval: (0.6, 0.7)
Precision: 1.0
Recall: 0.1

Computing the msd similarity matrix...
Done computing similarity matrix.
Threshold Interval: (0.7, 0.8)
Precision: 1.0
Recall: 0.1

Computing the msd similarity matrix...
Done computing similarity matrix.


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Threshold Interval: (0.8, 0.9)
Precision: 1.0
Recall: 0.1

Computing the msd similarity matrix...
Done computing similarity matrix.
Threshold Interval: (0.9, 1.0)
Precision: 1.0
Recall: 0.1

Best Threshold Interval: (0.5, 0.6)
Best Precision: 1.0
Best Recall: 0.1
F1-Score: 0.18181818181818182


# **Question 5**

In [ ]:
# Choose 10 random items
random_items = random.sample(list(ratings_df_600['movie_id'].unique()), 10)
# Randomly remove one rating for each item
for item in random_items:
    item_ratings = ratings_df_600[ratings_df_600['movie_id'] == item]
    if len(item_ratings) > 0:
        random_rating = random.choice(item_ratings.index)
        ratings_df_600_q5 = ratings_df_600.drop(random_rating)

In [ ]:
# Create a Surprise Dataset from the modified ratings DataFrame
reader = Reader(rating_scale=(1, 5))
data_sup_q5 = Dataset.load_from_df(ratings_df_600_q5[['user_id', 'movie_id', 'rating']], reader)

In [ ]:
# Build the full training set
trainset_itembased = data_sup_q5.build_full_trainset()

In [ ]:
# Create the KNNBasic model
model = KNNBasic( sim_options={'user_based': False})

In [ ]:
# Train the model on the full training set
model.fit(trainset_itembased)

Computing the msd similarity matrix...
Done computing similarity matrix.


In [ ]:
# Initialize lists to store actual and predicted ratings
actual_ratings = []
predicted_ratings = []

In [ ]:
# Iterate over the random items
for item in random_items:
    # Get the item's ratings
    item_ratings = ratings_df_600_q5[ratings_df_600_q5['movie_id'] == item]

    if len(item_ratings) > 0:
        # Get the user ID of the removed rating
        removed_user = item_ratings['user_id'].values[0]

        # Get the actual rating for the removed user-item pair
        actual_rating = item_ratings['rating'].values[0]
        actual_ratings.append(actual_rating)

        # Predict the rating for the removed user-item pair based on similar items
        predicted_rating = model.predict(trainset_itembased.to_inner_uid(removed_user), trainset_itembased.to_inner_iid(item),
                                         verbose=False).est
        predicted_ratings.append(predicted_rating)

In [ ]:
print(actual_ratings)

[5, 4, 4, 5, 4, 4, 3, 5, 5, 4]


In [ ]:
print(predicted_ratings)

[3.977151496577971, 3.4210247534879903, 3.785714285714285, 3.312794608062914, 2.9889518523618244, 3.5069466959740225, 3.4981639252824497, 3.5736059560099265, 3.290827615652893, 2.952762958923318]


In [ ]:
# Convert actual_ratings and predicted_ratings to numpy arrays
actual_ratings = np.array(actual_ratings)
predicted_ratings = np.array(predicted_ratings)

# Calculate MAE and RMSE
mae = np.mean(np.abs(predicted_ratings - actual_ratings))
rmse = np.sqrt(np.mean((predicted_ratings - actual_ratings) ** 2))

In [ ]:
print("MAE:", mae)
print("RMSE:", rmse)

MAE: 0.9688383702517307
RMSE: 1.088121070525213


# **Question 6**

# **based on k**

In [ ]:
# Initialize lists to store MAE and RMSE for each k
mae_values = []
rmse_values = []

# Initialize variables to store the best k and corresponding MAE/RMSE values
best_k = None
best_mae = float('inf')
best_rmse = float('inf')

In [ ]:
# Try different values of k
for k in [5, 15, 30, 50, 100]:
    # Create the KNNBasic model with the current k value
    model = KNNBasic(k=k, sim_options={'user_based': False})

    # Train the model on the full training set
    model.fit(trainset_itembased)

    # Initialize lists to store actual and predicted ratings
    actual_ratings = []
    predicted_ratings = []

    # Iterate over the random users
    for item in random_items:
        # Get the item's ratings
        item_ratings = ratings_df_600_q5[ratings_df_600_q5['movie_id'] == item]

        if len(item_ratings) > 0:
            # Get the user ID of the removed rating
            removed_user = item_ratings['user_id'].values[0]

            # Get the actual rating for the removed user-item pair
            actual_rating = item_ratings['rating'].values[0]
            actual_ratings.append(actual_rating)

            # Predict the rating for the removed user-item pair based on similar items
            predicted_rating = model.predict(trainset_itembased.to_inner_uid(removed_user), trainset_itembased.to_inner_iid(item),
                                         verbose=False).est
            predicted_ratings.append(predicted_rating)


    # Convert actual_ratings and predicted_ratings to numpy arrays
    actual_ratings = np.array(actual_ratings)
    predicted_ratings = np.array(predicted_ratings)

    # Calculate MAE and RMSE
    mae = np.mean(np.abs(predicted_ratings - actual_ratings))
    rmse = np.sqrt(np.mean((predicted_ratings - actual_ratings) ** 2))

    # Print the MAE and RMSE for the current k
    print(f"k = {k}")
    print("MAE:", mae)
    print("RMSE:", rmse)
    print("-----------------------------")

    # Check if current k gives lower MAE and RMSE
    if mae < best_mae and rmse < best_rmse:
        best_k = k
        best_mae = mae
        best_rmse = rmse

# Print the best k value, MAE, and RMSE
print("Best k:", best_k)
print("Best MAE:", best_mae)
print("Best RMSE:", best_rmse)

Computing the msd similarity matrix...
Done computing similarity matrix.
k = 5
MAE: 1.1326088811124149
RMSE: 1.2960101644026878
-----------------------------
Computing the msd similarity matrix...
Done computing similarity matrix.
k = 15
MAE: 1.0226992448814962
RMSE: 1.238678620975884
-----------------------------
Computing the msd similarity matrix...
Done computing similarity matrix.
k = 30
MAE: 0.970064178035835
RMSE: 1.1709729634492743
-----------------------------
Computing the msd similarity matrix...
Done computing similarity matrix.
k = 50
MAE: 0.925717307003028
RMSE: 1.1377820631347404
-----------------------------
Computing the msd similarity matrix...
Done computing similarity matrix.
k = 100
MAE: 0.9519636033429556
RMSE: 1.1759477897769224
-----------------------------
Best k: 50
Best MAE: 0.925717307003028
Best RMSE: 1.1377820631347404


## Calculate Precision, Recall and F1-score for Item-based collaborative filtering based on k.

In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
from surprise import KNNBasic
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import train_test_split

# Load MovieLens ratings dataset
ratings_df = pd.read_csv('movielens_ratings.csv')
ratings_df = ratings_df.sample(frac=0.3, random_state=42)

# Create Surprise Dataset
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings_df[['user_id', 'movie_id', 'rating']], reader)

# Train-test split
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# Initialize lists to store results for different K values
results = []

# Iterate over different K values
for k in [5, 15, 30, 50, 100]:
    # Create the KNNBasic model with the current k value
    model = KNNBasic(k=k, sim_options={'user_based': False})  # Use item-based collaborative filtering

    # Train the model on the full training set
    model.fit(trainset)

    # Initialize lists to store relevant items and recommended items
    relevant_items = []
    recommended_items = []

    # Iterate over the test set
    for user_id, movie_id, rating in testset:
        # Get the relevant items (items rated 4 or higher by the user in the test set)
        if rating >= 4:
            relevant_items.append(movie_id)

        # Predict the rating for the item based on the neighbors
        predicted_rating = model.predict(user_id, movie_id).est

        # Recommend the item if the predicted rating is 4 or higher
        if predicted_rating >= 4:
            recommended_items.append(movie_id)

    # Calculate precision and recall
    intersection = len(set(recommended_items) & set(relevant_items))
    precision = intersection / len(recommended_items) if len(recommended_items) > 0 else 0.0
    recall = intersection / len(relevant_items)
    F1 = 2 * (precision * recall) / (precision + recall)

    # Store the results for this K value
    results.append((k, precision, recall, F1))

# Sort the results based on F1-score in descending order
results.sort(key=lambda x: x[3], reverse=True)

# Print precision, recall, and F1-score for each K value
for k, precision, recall, F1 in results:
    print("K =", k)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1-Score:", F1)
    print()

# Print the best precision, recall, and F1-score value
best_k, best_precision, best_recall, best_F1 = results[0]
print("Best K =", best_k)
print("Best Precision:", best_precision)
print("Best Recall:", best_recall)
print("Best F1-Score:", best_F1)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
K = 100
Precision: 0.17071590540978296
Recall: 0.06123634673483616
F1-Score: 0.09013939964081073

K = 5
Precision: 0.1335527455395808
Recall: 0.06719149430629794
F1-Score: 0.08940339756102274

K = 50
Precision: 0.15928262025989817
Recall: 0.060887752730653034
F1-Score: 0.08809869070886663

K = 30
Precision: 0.15280322462440454
Recall: 0.0605682082268185
F1-Score: 0.08675029644885476

K = 15
Precision: 0.1462570914625709
Recall: 0.06141064373692773
F1-Score: 0.08650108433241949

Best K = 100
Best Precision: 0.17071590540978296
Best Recall: 0.06123634673483616
Best F1-Score: 0.09013939964081073


# **based on threshold**

In [ ]:
# Define the range of threshold intervals
threshold_intervals = [(0.5, 0.6), (0.6, 0.7), (0.7, 0.8), (0.8, 0.9), (0.9, 1.0)]

In [ ]:
# Initialize variables to store the best threshold, MAE, and RMSE
best_threshold = None
best_mae = float('inf')
best_rmse = float('inf')

In [ ]:
# Iterate over the threshold intervals
for threshold_interval in threshold_intervals:
    # Extract the lower and upper bounds of the threshold interval
    lower_bound, upper_bound = threshold_interval

    # Create a KNNBasic model with the current similarity threshold interval
    model = KNNBasic(sim_options={'user_based': False})

    # Train the model on the full dataset
    model.fit(trainset_itembased)

    # Initialize lists to store actual and predicted ratings
    actual_ratings = []
    predicted_ratings = []

    # Iterate over the random items
    for item in random_items:
        # Get the ratings for the current item
        item_ratings = ratings_df_600_q5[ratings_df_600_q5['movie_id'] == item]

        # Randomly select a user's rating for the current item to remove
        removed_rating = random.choice(item_ratings['rating'].tolist())

        # Get the actual rating for the removed item
        actual_rating = removed_rating

        # Calculate Pearson correlation between the current item and other items
        correlations = []
        for other_item in ratings_df_600_q5['movie_id'].unique():
            if other_item != item:
                other_item_ratings = ratings_df_600_q5[ratings_df_600_q5['movie_id'] == other_item]
                common_ratings = item_ratings.merge(other_item_ratings, on='user_id')
                if len(common_ratings) > 1 and np.var(common_ratings['rating_x']) > 0 and np.var(common_ratings['rating_y']) > 0:
                    correlation = pearsonr(common_ratings['rating_x'], common_ratings['rating_y'])[0]
                    correlations.append(correlation)

        # Filter neighbors based on similarity threshold
        filtered_neighbors = [ratings_df_600_q5['movie_id'].unique()[i] for i, correlation in enumerate(correlations)
                              if lower_bound <= correlation <= upper_bound]

        # Predict the rating based on the filtered neighbors
        if item in ratings_df_600_q5['movie_id'].unique() and len(filtered_neighbors) > 0:
            predicted_rating = np.mean([model.predict(user, item).est for user in filtered_neighbors])
        else:
            predicted_rating = np.nan

        # Append the actual and predicted ratings to the lists
        actual_ratings.append(actual_rating)
        predicted_ratings.append(predicted_rating)

    # Convert actual_ratings and predicted_ratings to numpy arrays
    actual_ratings = np.array(actual_ratings)
    predicted_ratings = np.array(predicted_ratings)

    # Calculate MAE and RMSE
    mae = np.mean(np.abs(predicted_ratings - actual_ratings))
    rmse = np.sqrt(np.mean((predicted_ratings - actual_ratings) ** 2))

    # Check if current threshold interval gives lower MAE and RMSE
    if mae < best_mae and rmse < best_rmse:
        best_threshold = threshold_interval
        best_mae = mae
        best_rmse = rmse

    # Print the MAE and RMSE for the current threshold interval
    print(f"Threshold Interval: {threshold_interval}")
    print("MAE:", mae)
    print("RMSE:", rmse)
    print()

# Print the best threshold interval, MAE, and RMSE
print("Best Threshold Interval:", best_threshold)
print("Best MAE:", best_mae)
print("Best RMSE:", best_rmse)

Computing the msd similarity matrix...
Done computing similarity matrix.
Threshold Interval: (0.5, 0.6)
MAE: 0.9603825191107758
RMSE: 1.2694832636575155

Computing the msd similarity matrix...
Done computing similarity matrix.
Threshold Interval: (0.6, 0.7)
MAE: 1.2566903959592348
RMSE: 1.5201543619312323

Computing the msd similarity matrix...
Done computing similarity matrix.
Threshold Interval: (0.7, 0.8)
MAE: 1.023826950300771
RMSE: 1.111398112545164

Computing the msd similarity matrix...
Done computing similarity matrix.
Threshold Interval: (0.8, 0.9)
MAE: 1.034777776352649
RMSE: 1.2015860617656238

Computing the msd similarity matrix...
Done computing similarity matrix.
Threshold Interval: (0.9, 1.0)
MAE: 0.8713997759295735
RMSE: 1.2449384465044666

Best Threshold Interval: (0.7, 0.8)
Best MAE: 0.925717307003028
Best RMSE: 1.1377820631347404


# Calculate Precision, Recall, and F1-Score for Item-based collaborative filtering based on threshold

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import pearsonr
from surprise import KNNBasic

# Load MovieLens ratings dataset
ratings_df = pd.read_csv('movielens_ratings.csv')
ratings_df = ratings_df.sample(frac=0.3, random_state=42)

# Define the range of threshold intervals
threshold_intervals = [(0.5, 0.6), (0.6, 0.7), (0.7, 0.8), (0.8, 0.9), (0.9, 1.0)]

# Initialize variables to store the best threshold, MAE, RMSE, precision, recall, and F1-score
best_threshold = None
best_mae = float('inf')
best_rmse = float('inf')
best_precision = 0.0
best_recall = 0.0
best_f1_score = 0.0

# Iterate over the threshold intervals
for threshold_interval in threshold_intervals:
    # Extract the lower and upper bounds of the threshold interval
    lower_bound, upper_bound = threshold_interval

    # Create a KNNBasic model with the current similarity threshold interval
    model = KNNBasic(sim_options={'user_based': False})

    # Train the model on the full dataset
    model.fit(trainset_itembased)

    # Initialize lists to store actual and predicted ratings
    actual_ratings = []
    predicted_ratings = []
    relevant_items = []
    recommended_items = []

    # Iterate over the random items
    for item in random_items:
        # Get the ratings for the current item
        item_ratings = ratings_df[ratings_df['movie_id'] == item]

        # Randomly select a user's rating for the current item to remove
        removed_rating = random.choice(item_ratings['rating'].tolist())

        # Get the actual rating for the removed item
        actual_rating = removed_rating

        # Calculate Pearson correlation between the current item and other items
        correlations = []
        for other_item in ratings_df['movie_id'].unique():
            if other_item != item:
                other_item_ratings = ratings_df[ratings_df['movie_id'] == other_item]
                common_ratings = item_ratings.merge(other_item_ratings, on='user_id')
                if len(common_ratings) > 1 and np.var(common_ratings['rating_x']) > 0 and np.var(common_ratings['rating_y']) > 0:
                    correlation = pearsonr(common_ratings['rating_x'], common_ratings['rating_y'])[0]
                    correlations.append(correlation)

        # Filter neighbors based on similarity threshold
        filtered_neighbors = [ratings_df['movie_id'].unique()[i] for i, correlation in enumerate(correlations)
                              if lower_bound <= correlation <= upper_bound]

        # Predict the rating based on the filtered neighbors
        if item in ratings_df['movie_id'].unique() and len(filtered_neighbors) > 0:
            predicted_rating = np.mean([model.predict(user, item).est for user in filtered_neighbors])
        else:
            predicted_rating = np.nan

        # Append the actual and predicted ratings to the lists
        actual_ratings.append(actual_rating)
        predicted_ratings.append(predicted_rating)

        # Append the relevant and recommended items to the lists
        relevant_items.append(item)
        recommended_items.append(item) if predicted_rating >= 4 else None

    # Convert actual_ratings and predicted_ratings to numpy arrays
    actual_ratings = np.array(actual_ratings)
    predicted_ratings = np.array(predicted_ratings)

    # Calculate MAE and RMSE
    mae = np.mean(np.abs(predicted_ratings - actual_ratings))
    rmse = np.sqrt(np.mean((predicted_ratings - actual_ratings) ** 2))

    # Calculate precision and recall
    intersection = len(set(recommended_items) & set(relevant_items))
    precision = intersection / len(recommended_items) if len(recommended_items) > 0 else 0.0
    recall = intersection / len(relevant_items) if len(relevant_items) > 0 else 0.0

    # Calculate F1-score
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0

    # Check if current threshold interval gives lower MAE and RMSE and higher precision, recall, and F1-score
    if mae < best_mae and rmse < best_rmse and precision > best_precision and recall > best_recall and f1_score > best_f1_score:
        best_threshold = threshold_interval
        best_mae = mae
        best_rmse = rmse
        best_precision = precision
        best_recall = recall
        best_f1_score = f1_score
        # Print the MAEand RMSE for the current threshold interval, as well as precision, recall, and F1-score:
    #print(f"Threshold Interval: {threshold_interval}")
    #print("MAE:", mae)
    #print("RMSE:", rmse)
    #print("Precision:", precision)
    #print("Recall:", recall)
    #print("F1-score:", f1_score)
    #print()

# Print the best threshold interval, MAE, RMSE, precision, recall, and F1-score
print("Best Threshold Interval:", best_threshold)
print("Best MAE:", best_mae)
print("Best RMSE:", best_rmse)
print("Best Precision:", best_precision)
print("Best Recall:", best_recall)
print("Best F1-score:", best_f1_score)



Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Best Threshold Interval: (0.5, 0.6)
Best MAE: 1.1985449238793235
Best RMSE: 1.324398719332224
Best Precision: 1.0
Best Recall: 0.2
Best F1-score: 0.33333333333333337


# **Question 7**

**Recommendations based on best values of K**

In [ ]:
# Define the number of items to recommend
num_recommendations = 5

# Define the value of K for user-based collaborative filtering
user_based_K = 15

# Define the value of K for item-based collaborative filtering
item_based_K = 100

# Randomly select a user
random_user = random.choice(ratings_df_600['user_id'].unique())

# Create a defaultdict to store the user's ratings
user_ratings = defaultdict(int)

# Iterate over the ratings and populate the user_ratings dictionary
for _, row in ratings_df_600[ratings_df_600['user_id'] == random_user].iterrows():
    movie_id = row['movie_id']
    rating = row['rating']
    user_ratings[movie_id] = rating

# Create a Reader object to parse the ratings DataFrame
reader = Reader(rating_scale=(1, 5))

# Load the ratings DataFrame into a Surprise Dataset
data = Dataset.load_from_df(ratings_df_600[['user_id', 'movie_id', 'rating']], reader)

# Build the full trainset
trainset = data.build_full_trainset()

# Create the KNNBasic model with user-based collaborative filtering
user_based_model = KNNBasic(k=user_based_K, sim_options={'user_based': True})

# Train the user-based model on the full training set
user_based_model.fit(trainset)

# Get the inner user ID for the random user
inner_user_id = trainset.to_inner_uid(random_user)

# Use the trained user-based model to get the top-K similar users to the random user
similar_users = user_based_model.get_neighbors(inner_user_id, k=user_based_K)

# Create a defaultdict to store the aggregated ratings from similar users
aggregated_ratings = defaultdict(float)
similar_users_count = 0

# Iterate over the similar users for user-based collaborative filtering
for user_id in similar_users:
    # Convert from inner user ID to raw user ID
    raw_user_id = trainset.to_raw_uid(user_id)

    # Create a defaultdict to store the ratings of the similar user
    similar_user_ratings = defaultdict(int)

    # Iterate over the ratings and populate the similar_user_ratings dictionary
    for _, row in ratings_df_600[ratings_df_600['user_id'] == raw_user_id].iterrows():
        movie_id = row['movie_id']
        rating = row['rating']
        similar_user_ratings[movie_id] = rating

    # If the similar user has ratings
    if similar_user_ratings:
        # Iterate over the similar user's ratings
        for movie_id, rating in similar_user_ratings.items():
            # Add the rating to the aggregated ratings
            aggregated_ratings[movie_id] += rating

        similar_users_count += 1

# Calculate the average ratings from similar users for user-based collaborative filtering
average_user_ratings = {movie_id: rating / similar_users_count for movie_id, rating in aggregated_ratings.items()}

# Sort the average user ratings in descending order
sorted_user_ratings = sorted(average_user_ratings.items(), key=lambda x: x[1], reverse=True)

# Get the top-N recommended items for user-based collaborative filtering
user_based_recommendations = [item_id for item_id, _ in sorted_user_ratings[:num_recommendations]]

# Create the KNNBasic model with item-based collaborative filtering
item_based_model = KNNBasic(k=item_based_K, sim_options={'user_based': False})

# Train the item-based model on the full training set
item_based_model.fit(trainset)

# Get the inner item ID for the items rated by the random user
inner_item_ids = [trainset.to_inner_iid(movie_id) for movie_id in user_ratings.keys()]

# Use the trained item-based model to get the top-K similar items for the rated items
similar_items = []
for inner_item_id in inner_item_ids:
    similar_items.extend(item_based_model.get_neighbors(inner_item_id, k=item_based_K))

# Filter out the items that have already been rated by the user
similar_items = [item_id for item_id in similar_items if trainset.to_raw_iid(item_id) not in user_ratings]

# Create a defaultdict to store the aggregated ratings from similar items
aggregated_item_ratings = defaultdict(float)
similar_items_count = 0

# Iterate over the similar items for item-based collaborative filtering
for item_id in similar_items:
    # Convert from inner item ID to raw item ID
    raw_item_id = trainset.to_raw_iid(item_id)

    # Get the rating of the similar item
    rating = user_ratings.get(raw_item_id, 0)

    # Add the rating to the aggregated ratings
    aggregated_item_ratings[raw_item_id] += rating

    similar_items_count += 1

# Calculate the average ratings from similar items for```python
# Calculate the average ratings from similar items for item-based collaborative filtering
average_item_ratings = {item_id: rating / similar_items_count for item_id, rating in aggregated_item_ratings.items()}

# Sort the average item ratings in descending order
sorted_item_ratings = sorted(average_item_ratings.items(), key=lambda x: x[1], reverse=True)

# Get the top-N recommended items for item-based collaborative filtering
item_based_recommendations = [item_id for item_id, _ in sorted_item_ratings[:num_recommendations]]

# Print the recommendations for user-based collaborative filtering
print(f"Recommendations for User {random_user} (User-based Collaborative Filtering):")
for item_id in user_based_recommendations:
    print(f"- Movie ID: {item_id}")

# Print the recommendations for item-based collaborative filtering
print(f"\nRecommendations for User {random_user} (Item-based Collaborative Filtering):")
for item_id in item_based_recommendations:
    print(f"- Movie ID: {item_id}")

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Recommendations for User 2447 (User-based Collaborative Filtering):
- Movie ID: 593
- Movie ID: 2858
- Movie ID: 2028
- Movie ID: 1617
- Movie ID: 318

Recommendations for User 2447 (Item-based Collaborative Filtering):
- Movie ID: 2544
- Movie ID: 3282
- Movie ID: 1871
- Movie ID: 1058
- Movie ID: 1471


**Recommendations based on best threshold intervals**

In [ ]:
# Define the number of items to recommend
num_recommendations = 5

# Define the threshold intervals for user-based and item-based collaborative filtering
user_based_threshold_interval = (0.7, 0.8)
item_based_threshold_interval = (0.8, 0.9)

# Randomly select a user
random_user = random.choice(ratings_df_600['user_id'].unique())

# Create a defaultdict to store the user's ratings
user_ratings = defaultdict(int)

# Iterate over the ratings and populate the user_ratings dictionary
for _, row in ratings_df_600[ratings_df_600['user_id'] == random_user].iterrows():
    movie_id = row['movie_id']
    rating = row['rating']
    user_ratings[movie_id] = rating

# Create a Reader object to parse the ratings DataFrame
reader = Reader(rating_scale=(1, 5))

# Load the ratings DataFrame into a Surprise Dataset
data = Dataset.load_from_df(ratings_df_600[['user_id', 'movie_id', 'rating']], reader)

# Build the full trainset
trainset = data.build_full_trainset()

# Create the KNNBasic model with user-based collaborative filtering
user_based_model = KNNBasic(sim_options={'user_based': True})

# Train the user-based model on the full training set
user_based_model.fit(trainset)

# Get the inner user ID for the random user
inner_user_id = trainset.to_inner_uid(random_user)

# Use the trained user-based model to get the similarities between the random user and other users
user_similarities = user_based_model.sim[inner_user_id]

# Create a defaultdict to store the aggregated ratings from similar users
aggregated_ratings_user = defaultdict(float)
similar_users_count = 0

# Iterate over the user similarities for user-based collaborative filtering
for user_id, similarity in enumerate(user_similarities):
    # Convert from inner user ID to raw user ID
    raw_user_id = trainset.to_raw_uid(user_id)

    # Create a defaultdict to store the ratings of the similar user
    similar_user_ratings = defaultdict(int)

    # Iterate over the ratings and populate the similar_user_ratings dictionary
    for _, row in ratings_df_600[ratings_df_600['user_id'] == raw_user_id].iterrows():
        movie_id = row['movie_id']
        rating = row['rating']
        similar_user_ratings[movie_id] = rating

    # Calculate the intersection of movies rated by the random user and the similar user
    common_movies_user = set(user_ratings.keys()) & set(similar_user_ratings.keys())


    # If there are at least two common movies
    if len(common_movies_user) >= 2:
        # Get the ratings for the common movies
        user_ratings_common = [user_ratings[movie_id] for movie_id in common_movies_user]
        similar_user_ratings_common = [similar_user_ratings[movie_id] for movie_id in common_movies_user]

        # Calculate the Pearson correlation between the random user and the similar user
        correlation_user = pearsonr(user_ratings_common, similar_user_ratings_common)[0]

        # Check if the similarity is within the specified threshold interval
        if user_based_threshold_interval[0] <= correlation_user <= user_based_threshold_interval[1]:
            # Iterate over the similar user's ratings
            for movie_id, rating in similar_user_ratings.items():
                # Add the rating to the aggregated ratings
                aggregated_ratings_user[movie_id] += rating

            similar_users_count += 1

# Calculate the average ratings from similar users for user-based collaborative filtering
average_user_ratings = {movie_id: rating / similar_users_count for movie_id, rating in aggregated_ratings_user.items()}

# Sort the average user ratings in descending order
sorted_user_ratings = sorted(average_user_ratings.items(), key=lambda x: x[1], reverse=True)

# Get the top-N recommended items for user-based collaborative filtering
user_based_recommendations = [item_id for item_id, _ in sorted_user_ratings[:num_recommendations]]

# Create the KNNBasic model with item-based collaborative filtering
item_based_model = KNNBasic(sim_options={'user_based': False})

# Train the item-based model on the full training set
item_based_model.fit(trainset)

# Get the inner item ID for the items rated by the random user
inner_item_ids = [trainset.to_inner_iid(movie_id) for movie_id in user_ratings.keys()]

# Use the trained item-based model to get the similarities between the items rated by the random user and other items
item_similarities = item_based_model.compute_similarities()

# Create a defaultdict to store the aggregated ratings from similar items
aggregated_ratings_item = defaultdict(float)
similar_items_count = 0

# Iterate over the item similarities for item-based collaborative filtering
for inner_item_id, similarity_row in enumerate(item_similarities):
    # Convert from inner item ID to raw item ID
    raw_item_id = trainset.to_raw_iid(inner_item_id)

    # Check if the item is rated by the random user
    if raw_item_id in user_ratings:
        # Get the similarity values for the items rated by the random user
        item_similarities_ratings = similarity_row[inner_item_ids]

        # Check if any items are sufficiently similar based on the threshold
        if any(item_based_threshold_interval[0] <= sim <= item_based_threshold_interval[1] for sim in item_similarities_ratings):
            # Iterate over the similar items' ratings
            for movie_id, rating in ratings_df_600[ratings_df_600['movie_id'] == raw_item_id][['movie_id', 'rating']].values:
                # Add the rating to the aggregated ratings
                aggregated_ratings_item[movie_id] += rating

            similar_items_count += 1

# Calculate the average ratings from similar items for item-based collaborative filtering
average_item_ratings = {movie_id: rating / similar_items_count for movie_id, rating in aggregated_ratings_item.items()}

# Sort the average item ratings in descending order
sorted_item_ratings = sorted(average_item_ratings.items(), key=lambda x: x[1], reverse=True)

# Get the top-N recommended items for item-based collaborative filtering
item_based_recommendations = [item_id for item_id, _ in sorted_item_ratings[:num_recommendations]]

# Print the recommended items for user-based collaborative filtering
print(f"User-Based Recommendations for user {random_user}:")
for item_id in user_based_recommendations:
    print("movie id :", item_id)

# Print the recommended items for item-based collaborative filtering
print(f"Item-Based Recommendations for user {random_user}:")
for item_id in item_based_recommendations:
    print("movie id :" , item_id)

Computing the msd similarity matrix...
Done computing similarity matrix.


/usr/local/lib/python3.10/dist-packages/scipy/stats/_stats_py.py:4781: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(msg))


Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
User-Based Recommendations for user 53:
movie id : 2858
movie id : 858
movie id : 260
movie id : 2997
movie id : 2396
Item-Based Recommendations for user 53:
movie id : 480
movie id : 318
movie id : 2997
movie id : 457
movie id : 1036


# **Question 8**

In [ ]:
# Define the number of items to recommend
num_recommendations = 5

# Randomly select a user
random_user = random.choice(ratings_df_600['user_id'].unique())

# Create a Reader object to parse the ratings DataFrame
reader = Reader(rating_scale=(1, 5))

# Load the ratings DataFrame into a Surprise Dataset
data = Dataset.load_from_df(ratings_df_600[['user_id', 'movie_id', 'rating']], reader)

# Build the full trainset
trainset = data.build_full_trainset()

# User-Based Collaborative Filtering
start_time_user_based = time.time()

# Create the KNNBasic model with user-based collaborative filtering
user_based_model = KNNBasic(k=30, sim_options={'user_based': True})

# Train the user-based model on the full training set
user_based_model.fit(trainset)

# Get the inner user ID for the random user
inner_user_id = trainset.to_inner_uid(random_user)

# Use the trained user-based model to get recommendations
user_based_recommendations = user_based_model.get_neighbors(inner_user_id, k=num_recommendations)

end_time_user_based = time.time()
user_based_time = end_time_user_based - start_time_user_based

# Item-Based Collaborative Filtering
start_time_item_based = time.time()

# Create the KNNBasic model with item-based collaborative filtering
item_based_model = KNNBasic(k=30, sim_options={'user_based': False})

# Train the item-based model on the full training set
item_based_model.fit(trainset)

# Use the trained item-based model to get item similarities
item_similarities = item_based_model.compute_similarities()

# Get the top-N items similar to those rated by the random user
item_based_recommendations = []

for inner_item_id in inner_item_ids:
    # Find the most similar items
    item_similarities_ratings = item_similarities[inner_item_id]
    similar_items = sorted(enumerate(item_similarities_ratings), key=lambda x: x[1], reverse=True)[:num_recommendations]

    # Extract the item IDs
    similar_item_ids = [trainset.to_raw_iid(item_id) for item_id, _ in similar_items]

    item_based_recommendations.extend(similar_item_ids)

    # Limit the recommendations to 5 for each technique
    item_based_recommendations = item_based_recommendations[:num_recommendations]

end_time_item_based = time.time()
item_based_time = end_time_item_based - start_time_item_based

# Print the recommended items for user-based collaborative filtering
print(f"User-Based Recommendations for user {random_user}: {user_based_recommendations}")
print(f"Time taken for user-based collaborative filtering: {user_based_time:.4f} seconds\n")

# Print the recommended items for item-based collaborative filtering
print(f"Item-Based Recommendations for user {random_user}: {item_based_recommendations}")
print(f"Time taken for item-based collaborative filtering: {item_based_time:.4f} seconds")

# Compare timestamps
if user_based_time < item_based_time:
    print("User-Based Collaborative Filtering is faster.")
elif user_based_time > item_based_time:
    print("Item-Based Collaborative Filtering is faster.")
else:
    print("Both methods took the same amount of time.")


Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
User-Based Recommendations for user 874: [2, 11, 12, 40, 48]
Time taken for user-based collaborative filtering: 19.7974 seconds

Item-Based Recommendations for user 874: [2987, 3056, 3640, 3154, 2679]
Time taken for item-based collaborative filtering: 18.9686 seconds
Item-Based Collaborative Filtering is faster.
